In [ ]:
# 目标：使用聊天模型将文本分类到标签中
# 分类以下标签
# 情绪
# 语言
# 风格（正式、非正式等）
# 涵盖的主题
# 政治倾向


In [ ]:
# 相关环境变量设置
import config_loader

config_loader.load_env()

In [ ]:
from pydantic import Field, BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI


# 打标签的提示词
tagging_prompt = ChatPromptTemplate.from_template(
    """
    从下面的文本中提取所需的信息。

    只提取` Classification `函数中提到的属性。

    文本：
    {input}
    """
)

class Classification(BaseModel):
    """
    定义一个 pydantic 模型
    有属性及类型定义
    llm会根据这个定义来输出结果
    """
    sentiment: str = Field(description="文本的情感")
    aggressiveness: int = Field(
        description="从1到10，对这段文本的攻击性进行评分"
    )
    language: str = Field(description="书写文本的语言")

# 配置谷歌聊天模型 并在后面添加 with_structured_output 限制，限制模型为结构化输出
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
).with_structured_output(Classification)

In [16]:
inp = "我的 bug 终于改完了！"
prompt = tagging_prompt.invoke({"input": inp})
response = llm.invoke(prompt)

# 原始输出 直接是 Classification 对象
print(response)
# 字典输出
print(response.model_dump())

sentiment='positive' aggressiveness=1 language='chinese'
{'sentiment': 'positive', 'aggressiveness': 1, 'language': 'chinese'}


In [28]:
# 上面可以正常判断了，但是输出的标签是随机的，有时候是中文，有时候是英文，并不稳定
# 下面介绍如何更精细化判断

from pydantic import Field, BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# 官方文档这个有警告，并且还会报错：TypeError: bad argument type for built-in operation
# 评论区说是 Gemini 系列模型的问题
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["高兴", "自然", "悲伤"])
    aggressiveness: int = Field(
        ...,
        description="描述语句的攻击性，数字越高越强",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["西班牙语", "英语", "法语", "德语", "意大利语", "中文"]
    )

In [ ]:
# 再把提示词一起拿过来 方便查看
tagging_prompt = ChatPromptTemplate.from_template(
    """
    从下面的文本中提取所需的信息。

    只提取` Classification `函数中提到的属性。

    文本：
    {input}
    """
)

# 重新定义大模型对象
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
).with_structured_output(Classification)
